In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from pyaxis import pyaxis
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
prices = pd.read_csv('Prices.csv')
quantity = pd.read_csv('Quantity.csv')
prices.sample(10)


,Jahr,Gemeinde_Nr,Gemeinde,Messkategorie,Wert,Einheit
1012,2022,29,Flurlingen,Grundgebuehr,269.25,CHF
2817,2011,85,Dänikon,Grundgebuehr,209.50,CHF
3434,2015,98,Schleinikon,35-L-Sack,1.80,CHF
2334,2011,66,Opfikon,35-L-Sack,1.80,CHF
785,2019,24,Buch a.I.,Grundgebuehr,100.00,CHF
4687,2004,155,Männedorf,35-L-Sack,1.60,CHF
6986,2005,249,Unterengstringen,35-L-Sack,2.00,CHF
1318,2015,35,Marthalen,35-L-Sack,1.50,CHF
4978,2012,172,Fehraltorf,Grundgebuehr,135.00,CHF
300,2021,7,Knonau,35-L-Sack,1.80,CHF


In [75]:
quantity.sample(5)

,Jahr,Gemeinde_Nr,Gemeinde,Abfallart,Wert,Einheit
22968,2015,161,Zollikon,Brennbare Abfälle und Sperrgut,2063.0,Tonnen
31225,2002,243,Dietikon,Grubengut,350.0,Tonnen
27079,2010,211,Altikon,Brennbare Abfälle und Sperrgut,125.0,Tonnen
25046,2021,192,Egg,Kunststoffe,43.0,Tonnen
17603,2017,114,Fischenthal,Karton,10.0,Tonnen


In [76]:
prices.merge(quantity,how='left')

,Jahr,Gemeinde_Nr,Gemeinde,Messkategorie,Wert,Einheit,Abfallart
0,2022,1,Aeugst a.A.,Grundgebuehr,204.63,CHF,NaN
1,2021,1,Aeugst a.A.,Grundgebuehr,204.63,CHF,NaN
2,2020,1,Aeugst a.A.,Grundgebuehr,204.63,CHF,NaN
3,2019,1,Aeugst a.A.,Grundgebuehr,204.63,CHF,NaN
4,2018,1,Aeugst a.A.,Grundgebuehr,205.20,CHF,NaN
...,...,...,...,...,...,...,...
7447,2004,298,Wiesendangen,35-L-Sack,1.80,CHF,NaN
7448,2003,298,Wiesendangen,35-L-Sack,1.80,CHF,NaN
7449,2002,298,Wiesendangen,35-L-Sack,1.80,CHF,NaN
7450,2001,298,Wiesendangen,35-L-Sack,1.80,CHF,NaN


In [77]:
ind = pd.read_excel('population_2.xlsx',skiprows=2)["Unnamed: 1"].values

In [78]:
l =[]
for i in ind:
    if i >200:
        print(i)
        k = i
    else: i =k
    l.append(k)


2000.0
2001.0
2002.0
2003.0
2004.0
2005.0
2006.0
2007.0
2008.0
2009.0
2010.0
2011.0
2012.0
2013.0
2014.0
2015.0
2016.0
2017.0
2018.0
2019.0
2020.0
2021.0


In [79]:
df =pd.read_excel('population_2.xlsx',skiprows=2)
df["IDX"] =l
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Population on 31 December,IDX
0,2000,2000.0,- Zürich,Citizenship (category) - total,Sex - total,1211647.0,2000.0
1,NaN,NaN,>> Bezirk Affoltern,Citizenship (category) - total,Sex - total,41455.0,2000.0
2,NaN,NaN,......0001 Aeugst am Albis,Citizenship (category) - total,Sex - total,1517.0,2000.0
3,NaN,NaN,......0002 Affoltern am Albis,Citizenship (category) - total,Sex - total,9933.0,2000.0
4,NaN,NaN,......0003 Bonstetten,Citizenship (category) - total,Sex - total,3896.0,2000.0
...,...,...,...,...,...,...,...
3885,Database:,NaN,NaN,NaN,NaN,NaN,2021.0
3886,"FSO - STAT-TAB / Federal Statistical Office, 2...",NaN,NaN,NaN,NaN,NaN,2021.0
3887,NaN,NaN,NaN,NaN,NaN,NaN,2021.0
3888,Internal reference code:,NaN,NaN,NaN,NaN,NaN,2021.0


In [80]:
df.tail(50)
end =3714 - 3675	
df = df[:-end-1]

In [81]:
df["Unnamed: 2"].value_counts()

- Zürich                    22
......0172 Fehraltorf       22
......0155 Männedorf        22
......0156 Meilen           22
......0157 Oetwil am See    22
                            ..
......0071 Wil (ZH)         22
......0072 Winkel           22
>> Bezirk Dielsdorf         22
......0081 Bachs            22
......0261 Zürich           22
Name: Unnamed: 2, Length: 175, dtype: int64

In [82]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Population on 31 December', 'IDX'],
      dtype='object')

In [83]:
df = df[['Unnamed: 2','Population on 31 December', 'IDX']]
df

,Unnamed: 2,Population on 31 December,IDX
0,- Zürich,1211647.0,2000.0
1,>> Bezirk Affoltern,41455.0,2000.0
2,......0001 Aeugst am Albis,1517.0,2000.0
3,......0002 Affoltern am Albis,9933.0,2000.0
4,......0003 Bonstetten,3896.0,2000.0
...,...,...,...
3845,......0249 Unterengstringen,4116.0,2021.0
3846,......0250 Urdorf,10222.0,2021.0
3847,......0251 Weiningen (ZH),4863.0,2021.0
3848,>> Bezirk Zürich,423193.0,2021.0
